In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
import torch
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import MNIST
from torchvision import transforms as tfr
import matplotlib.pyplot as plt

In [12]:
ts = tfr.Compose([
    tfr.ToTensor(),
    tfr.Normalize(0.5,0.5),
    tfr.Resize((64,64))
    
])
root ='data'
ds = MNIST(root, train= True, download=True,transform=ts)
test_df = MNIST(root, train=False, download=True, transform=ts)

In [13]:
ds[0][0].shape


torch.Size([1, 64, 64])

In [14]:
tr_length = int(len(ds) * 0.9)
val_length = int(len(ds) * 0.1)

tr_ds, val_ds = random_split(ds, [tr_length, val_length])

print(f"Train dataset has {len(tr_ds)} images.")
print(f"Validation dataset has {len(val_ds)} images.")
print(f"Test dataset has {len(test_df)} images.")

Train dataset has 54000 images.
Validation dataset has 6000 images.
Test dataset has 10000 images.


In [15]:
bs = 64
tr_dl = DataLoader(dataset = tr_ds, batch_size = bs, shuffle = True, drop_last = False, num_workers = 4)
val_dl = DataLoader(dataset = val_ds, batch_size = bs, shuffle = False, drop_last = False, num_workers = 4)
test_dl = DataLoader(dataset = test_df, batch_size = bs, shuffle = False, drop_last = False, num_workers = 4)

print(f"Train dataloaderda {len(tr_dl)} ta batch bor.") 
print(f"Validation dataloaderda {len(val_dl)} ta batch bor.")
print(f"Test dataloaderda {len(test_dl)} ta batch bor.") 

Train dataloaderda 844 ta batch bor.
Validation dataloaderda 94 ta batch bor.
Test dataloaderda 157 ta batch bor.


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
im_size = 64
in_chs = 1
out_chs = 8
ks = 3
pad = 1 
pool_size = 2
num_classes = 10

in_fs = (im_size // 32) * (im_size // 32)* out_chs * 8
print(in_fs)

256


In [17]:
class Model(torch.nn.Module):
    
    def __init__(self, in_chs, out_chs, in_fs, ks, pad, ps, n_cls): 
        super(Model, self).__init__()
        
        self.in_chs, self.out_chs, self.ks, self.pad, self.ps, self.n_cls = in_chs, out_chs, pad, ks, ps, n_cls
        
        self.conv_1 = torch.nn.Conv2d(in_channels =in_chs, out_channels =out_chs, kernel_size = ks, padding = pad)
        self.act_1 = torch.nn.ReLU()
        self.pool_1 = torch.nn.MaxPool2d(kernel_size = ps, stride = ps)
        
        self.conv_2 = torch.nn.Conv2d(in_channels =out_chs, out_channels =out_chs*2, kernel_size = ks, padding = pad)
        self.act_2 = torch.nn.ReLU()
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=ps, stride=ps)
        
        self.conv_3 = torch.nn.Conv2d(in_channels=out_chs*2, out_channels=out_chs*4, kernel_size=ks, padding=pad)
        self.act_3 = torch.nn.ReLU()
        self.pool_3 = torch.nn.MaxPool2d(kernel_size=ps, stride=ps)
        
        self.conv_4 = torch.nn.Conv2d(in_channels=out_chs*4, out_channels=out_chs*6, kernel_size=ks, padding=pad)
        self.pool_4 = torch.nn.MaxPool2d(kernel_size=ps, stride=ps) 
        self.act_4 = torch.nn.ReLU()
        
        self.conv_5 = torch.nn.Conv2d(in_channels=out_chs*6, out_channels=out_chs*8, kernel_size=ks, padding=pad)
        self.act_5 = torch.nn.ReLU()
        self.pool_5 = torch.nn.MaxPool2d(kernel_size=ps, stride=ps)
        
        self.fc = torch.nn.Linear(in_features = in_fs, out_features = in_fs//2)
        self.fc1 = torch.nn.Linear(in_features=in_fs//2, out_features=in_fs//4)
        self.out = torch.nn.Linear(in_features = in_fs//4, out_features = n_cls)
        self.softmax = torch.nn.Softmax(dim = 1)
        
    def forward(self, inp):
        
        conv_1 = self.conv_1(inp)
        act_1 = self.act_1(conv_1)
        out_1 = self.pool_1(act_1)
        
        conv_2 = self.conv_2(out_1)
        act_2 = self.act_2(conv_2)
        out_2 = self.pool_2(act_2)
        
        conv_3 = self.conv_3(out_2)
        act_3 = self.act_3(conv_3)
        out_3 = self.pool_3(act_3)
        
        conv_4 = self.conv_4(out_3)
        act_4 = self.act_4(conv_4)
        out_4 =self.pool_4(act_4)
        
        conv_5 = self.conv_5(out_4)
        act_5  = self.act_5(conv_5)
        out_5  = self.pool_5(act_5)
          
        
        bs = inp.shape[0]
        flat = out_5.view(bs, -1)
        #fc_1 = self.fc(flat)
        #fc_2 = self.fc1(fc_1)
        #out =self.softmax(fc_2)
        
        
        fc_1 = self.act_4(self.fc(flat))
        fc_2 = self.act_4(self.fc1(fc_1))
        out = self.softmax(self.out(fc_2))
        
        return self.softmax(out)
    
m = Model(in_chs = in_chs, out_chs = out_chs, in_fs = in_fs, ks = ks, pad = pad, ps = pool_size, n_cls = num_classes)
m

Model(
  (conv_1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_1): ReLU()
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_3): ReLU()
  (pool_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_4): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool_4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (act_4): ReLU()
  (conv_5): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_5): ReLU()
  (pool_5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=256, out_features=128, bias=True)
  (fc1): Linear(in_f

In [18]:
m = Model(in_chs = in_chs, out_chs = out_chs, in_fs = in_fs, ks = ks, pad = pad, ps = pool_size, n_cls = num_classes)

x = torch.rand(1, in_chs, im_size, im_size)
m(x)

tensor([[0.1000, 0.1007, 0.0998, 0.1007, 0.0994, 0.0997, 0.0994, 0.1003, 0.1010,
         0.0989]], grad_fn=<SoftmaxBackward0>)

In [19]:
def train_setup(m): return m.to("cuda"), 10, "cuda", torch.nn.CrossEntropyLoss(), torch.optim.Adam(params = m.parameters(), lr = 0.001)

In [20]:
m, epochs, device, loss_fn, optimizer = train_setup(m)

# 90% train 10% validation

In [21]:
from tqdm import tqdm

print("Train boshlandi...")
best_acc = 0
for epoch in range(epochs):
    
    epoch_loss, epoch_acc, total = 0, 0, 0
    for idx, batch in tqdm(enumerate(tr_dl)):
        ims, gts = batch
        ims, gts = ims.to(device), gts.to(device)
        
        # print(ims.shape)
        # print(gts.shape)
        total += ims.shape[0]
        
        preds = m(ims)
        _, pred_cls = torch.max(preds.data, dim = 1)
        # print(pred_cls)
        # print(gts)
        loss = loss_fn(preds, gts)
        # loss = loss_fn(preds, gts.unsqueeze(1))
        
        epoch_acc += (pred_cls == gts).sum().item()
        epoch_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    tr_loss = epoch_loss / len(tr_dl)
    print(f"Epoch {epoch + 1} train jarayoni tugadi")
    print(f"Epoch {epoch + 1} dagi train loss -> {(tr_loss):.3f}")
    print(f"Epoch {epoch + 1} dagi train accuracy -> {(epoch_acc / total):.3f}")
    
    m.eval()
    with torch.no_grad():
        val_epoch_loss, val_epoch_acc, val_total = 0, 0, 0
        for idx, batch in enumerate(val_dl):
            ims, gts = batch
            ims, gts = ims.to(device), gts.to(device)
            val_total += ims.shape[0]

            preds = m(ims)
            loss = loss_fn(preds, gts)
            _, pred_cls = torch.max(preds.data, dim = 1)
            val_epoch_acc += (pred_cls == gts).sum().item()
            val_epoch_loss += loss.item()
        
        val_acc = val_epoch_acc / val_total
        val_loss = val_epoch_loss / len(val_dl)
        
        print(f"Epoch {epoch + 1} validation jarayoni tugadi")
        print(f"Epoch {epoch + 1} dagi validation loss -> {(val_loss):.3f}")
        print(f"Epoch {epoch + 1} dagi validation accuracy -> {val_acc:.3f}")
        
        
        if val_acc > best_acc:
            os.makedirs("saved_models", exist_ok=True)
            best_acc = val_acc
            torch.save(m.state_dict(), f"saved_models/cifar_best_model_{val_acc:.3f}.pth")        


Train boshlandi...


844it [00:24, 33.99it/s]

Epoch 1 train jarayoni tugadi
Epoch 1 dagi train loss -> 2.230
Epoch 1 dagi train accuracy -> 0.605


Epoch 1 validation jarayoni tugadi
Epoch 1 dagi validation loss -> 2.218
Epoch 1 dagi validation accuracy -> 0.686


844it [00:20, 40.56it/s]

...

Epoch 9 train jarayoni tugadi
Epoch 9 dagi train loss -> 2.188
Epoch 9 dagi train accuracy -> 0.886


Epoch 9 validation jarayoni tugadi
Epoch 9 dagi validation loss -> 2.188
Epoch 9 dagi validation accuracy -> 0.888


844it [00:20, 41.30it/s]

Epoch 10 train jarayoni tugadi
Epoch 10 dagi train loss -> 2.188
Epoch 10 dagi train accuracy -> 0.887


Epoch 10 validation jarayoni tugadi
Epoch 10 dagi validation loss -> 2.189
Epoch 10 dagi validation accuracy -> 0.881


# 5% train 50% validation

In [22]:
47it [00:01, 43.28it/s]
Epoch 1 train jarayoni tugadi
Epoch 1 dagi train loss -> 2.282
Epoch 1 dagi train accuracy -> 0.283

Epoch 1 validation jarayoni tugadi
Epoch 1 dagi validation loss -> 2.262
Epoch 1 dagi validation accuracy -> 0.396
47it [00:01, 41.01it/s]
Epoch 2 train jarayoni tugadi
Epoch 2 dagi train loss -> 2.244
Epoch 2 dagi train accuracy -> 0.509

Epoch 2 validation jarayoni tugadi
Epoch 2 dagi validation loss -> 2.236
Epoch 2 dagi validation accuracy -> 0.563
47it [00:01, 42.47it/s]
Epoch 3 train jarayoni tugadi
Epoch 3 dagi train loss -> 2.234
Epoch 3 dagi train accuracy -> 0.580

Epoch 3 validation jarayoni tugadi
Epoch 3 dagi validation loss -> 2.236
Epoch 3 dagi validation accuracy -> 0.567
47it [00:01, 41.93it/s]
Epoch 4 train jarayoni tugadi
Epoch 4 dagi train loss -> 2.233
Epoch 4 dagi train accuracy -> 0.585

Epoch 4 validation jarayoni tugadi
Epoch 4 dagi validation loss -> 2.232
Epoch 4 dagi validation accuracy -> 0.588
47it [00:01, 42.14it/s]
Epoch 5 train jarayoni tugadi
Epoch 5 dagi train loss -> 2.232
Epoch 5 dagi train accuracy -> 0.588

Epoch 5 validation jarayoni tugadi
Epoch 5 dagi validation loss -> 2.232
Epoch 5 dagi validation accuracy -> 0.587
47it [00:01, 42.51it/s]
Epoch 6 train jarayoni tugadi
Epoch 6 dagi train loss -> 2.231
Epoch 6 dagi train accuracy -> 0.595

Epoch 6 validation jarayoni tugadi
Epoch 6 dagi validation loss -> 2.232
Epoch 6 dagi validation accuracy -> 0.587
47it [00:01, 42.40it/s]
Epoch 7 train jarayoni tugadi
Epoch 7 dagi train loss -> 2.225
Epoch 7 dagi train accuracy -> 0.635

Epoch 7 validation jarayoni tugadi
Epoch 7 dagi validation loss -> 2.221
Epoch 7 dagi validation accuracy -> 0.667
47it [00:01, 42.05it/s]
Epoch 8 train jarayoni tugadi
Epoch 8 dagi train loss -> 2.218
Epoch 8 dagi train accuracy -> 0.683

Epoch 8 validation jarayoni tugadi
Epoch 8 dagi validation loss -> 2.218
Epoch 8 dagi validation accuracy -> 0.686
47it [00:01, 42.04it/s]
Epoch 9 train jarayoni tugadi
Epoch 9 dagi train loss -> 2.210
Epoch 9 dagi train accuracy -> 0.741

Epoch 9 validation jarayoni tugadi
Epoch 9 dagi validation loss -> 2.209
Epoch 9 dagi validation accuracy -> 0.747
47it [00:01, 41.19it/s]
Epoch 10 train jarayoni tugadi
Epoch 10 dagi train loss -> 2.205
Epoch 10 dagi train accuracy -> 0.774

Epoch 10 validation jarayoni tugadi
Epoch 10 dagi validation loss -> 2.206
Epoch 10 dagi validation accuracy -> 0.767

SyntaxError: invalid syntax (209486628.py, line 1)

# 10% of training data

In [ ]:
10% of training data
94it [00:06, 14.92it/s]
Epoch 1 train jarayoni tugadi
Epoch 1 dagi train loss -> 2.303
Epoch 1 dagi train accuracy -> 0.105

Epoch 1 validation jarayoni tugadi
Epoch 1 dagi validation loss -> 2.303
Epoch 1 dagi validation accuracy -> 0.103
94it [00:02, 44.33it/s]
Epoch 2 train jarayoni tugadi
Epoch 2 dagi train loss -> 2.291
Epoch 2 dagi train accuracy -> 0.187

Epoch 2 validation jarayoni tugadi
Epoch 2 dagi validation loss -> 2.275
Epoch 2 dagi validation accuracy -> 0.296
94it [00:02, 43.69it/s]
Epoch 3 train jarayoni tugadi
Epoch 3 dagi train loss -> 2.274
Epoch 3 dagi train accuracy -> 0.303

Epoch 3 validation jarayoni tugadi
Epoch 3 dagi validation loss -> 2.274
Epoch 3 dagi validation accuracy -> 0.298
94it [00:02, 45.05it/s]
Epoch 4 train jarayoni tugadi
Epoch 4 dagi train loss -> 2.260
Epoch 4 dagi train accuracy -> 0.397

Epoch 4 validation jarayoni tugadi
Epoch 4 dagi validation loss -> 2.250
Epoch 4 dagi validation accuracy -> 0.473
94it [00:02, 33.10it/s]
Epoch 5 train jarayoni tugadi
Epoch 5 dagi train loss -> 2.248
Epoch 5 dagi train accuracy -> 0.482

Epoch 5 validation jarayoni tugadi
Epoch 5 dagi validation loss -> 2.248
Epoch 5 dagi validation accuracy -> 0.477
94it [00:02, 40.80it/s]
Epoch 6 train jarayoni tugadi
Epoch 6 dagi train loss -> 2.247
Epoch 6 dagi train accuracy -> 0.487

Epoch 6 validation jarayoni tugadi
Epoch 6 dagi validation loss -> 2.247
Epoch 6 dagi validation accuracy -> 0.487
94it [00:02, 42.32it/s]
Epoch 7 train jarayoni tugadi
Epoch 7 dagi train loss -> 2.246
Epoch 7 dagi train accuracy -> 0.491

Epoch 7 validation jarayoni tugadi
Epoch 7 dagi validation loss -> 2.247
Epoch 7 dagi validation accuracy -> 0.487
94it [00:02, 40.07it/s]
Epoch 8 train jarayoni tugadi
Epoch 8 dagi train loss -> 2.242
Epoch 8 dagi train accuracy -> 0.522

Epoch 8 validation jarayoni tugadi
Epoch 8 dagi validation loss -> 2.235
Epoch 8 dagi validation accuracy -> 0.565
94it [00:02, 42.05it/s]
Epoch 9 train jarayoni tugadi
Epoch 9 dagi train loss -> 2.226
Epoch 9 dagi train accuracy -> 0.628
Epoch 9 validation jarayoni tugadi
Epoch 9 dagi validation loss -> 2.222
Epoch 9 dagi validation accuracy -> 0.657
94it [00:02, 41.72it/s]
Epoch 10 train jarayoni tugadi
Epoch 10 dagi train loss -> 2.222
Epoch 10 dagi train accuracy -> 0.661

Epoch 10 validation jarayoni tugadi
Epoch 10 dagi validation loss -> 2.221
Epoch 10 dagi validation accuracy -> 0.665


# 20% train 10% validation

In [ ]:
188it [00:04, 45.20it/s]
Epoch 1 train jarayoni tugadi
Epoch 1 dagi train loss -> 2.282
Epoch 1 dagi train accuracy -> 0.248

Epoch 1 validation jarayoni tugadi
Epoch 1 dagi validation loss -> 2.264
Epoch 1 dagi validation accuracy -> 0.371
188it [00:05, 36.75it/s]
Epoch 2 train jarayoni tugadi
Epoch 2 dagi train loss -> 2.254
Epoch 2 dagi train accuracy -> 0.441

Epoch 2 validation jarayoni tugadi
Epoch 2 dagi validation loss -> 2.236
Epoch 2 dagi validation accuracy -> 0.565
188it [00:04, 43.92it/s]
Epoch 3 train jarayoni tugadi
Epoch 3 dagi train loss -> 2.234
Epoch 3 dagi train accuracy -> 0.576

Epoch 3 validation jarayoni tugadi
Epoch 3 dagi validation loss -> 2.232
Epoch 3 dagi validation accuracy -> 0.588
188it [00:04, 44.66it/s]
Epoch 4 train jarayoni tugadi
Epoch 4 dagi train loss -> 2.224
Epoch 4 dagi train accuracy -> 0.643

Epoch 4 validation jarayoni tugadi
Epoch 4 dagi validation loss -> 2.221
Epoch 4 dagi validation accuracy -> 0.665
188it [00:04, 41.10it/s]
Epoch 5 train jarayoni tugadi
Epoch 5 dagi train loss -> 2.209
Epoch 5 dagi train accuracy -> 0.745

Epoch 5 validation jarayoni tugadi
Epoch 5 dagi validation loss -> 2.207
Epoch 5 dagi validation accuracy -> 0.761
188it [00:04, 45.24it/s]
Epoch 6 train jarayoni tugadi
Epoch 6 dagi train loss -> 2.204
Epoch 6 dagi train accuracy -> 0.779

Epoch 6 validation jarayoni tugadi
Epoch 6 dagi validation loss -> 2.204
Epoch 6 dagi validation accuracy -> 0.776
188it [00:04, 43.89it/s]
Epoch 7 train jarayoni tugadi
Epoch 7 dagi train loss -> 2.203
Epoch 7 dagi train accuracy -> 0.787

Epoch 7 validation jarayoni tugadi
Epoch 7 dagi validation loss -> 2.205
Epoch 7 dagi validation accuracy -> 0.772
188it [00:04, 41.71it/s]
Epoch 8 train jarayoni tugadi
Epoch 8 dagi train loss -> 2.203
Epoch 8 dagi train accuracy -> 0.785

Epoch 8 validation jarayoni tugadi
Epoch 8 dagi validation loss -> 2.204
Epoch 8 dagi validation accuracy -> 0.779
188it [00:04, 44.80it/s]
Epoch 9 train jarayoni tugadi
Epoch 9 dagi train loss -> 2.197
Epoch 9 dagi train accuracy -> 0.831

Epoch 9 validation jarayoni tugadi
Epoch 9 dagi validation loss -> 2.192
Epoch 9 dagi validation accuracy -> 0.859
188it [00:04, 41.21it/s]
Epoch 10 train jarayoni tugadi
Epoch 10 dagi train loss -> 2.190
Epoch 10 dagi train accuracy -> 0.878

Epoch 10 validation jarayoni tugadi
Epoch 10 dagi validation loss -> 2.191
Epoch 10 dagi validation accuracy -> 0.871


# 50% training 10 validation

In [ ]:
469it [00:10, 42.86it/s]
Epoch 1 train jarayoni tugadi
Epoch 1 dagi train loss -> 2.217
Epoch 1 dagi train accuracy -> 0.700

Epoch 1 validation jarayoni tugadi
Epoch 1 dagi validation loss -> 2.196
Epoch 1 dagi validation accuracy -> 0.839
469it [00:11, 41.40it/s]
Epoch 2 train jarayoni tugadi
Epoch 2 dagi train loss -> 2.194
Epoch 2 dagi train accuracy -> 0.848

Epoch 2 validation jarayoni tugadi
Epoch 2 dagi validation loss -> 2.192
Epoch 2 dagi validation accuracy -> 0.861
469it [00:10, 43.24it/s]
Epoch 3 train jarayoni tugadi
Epoch 3 dagi train loss -> 2.191
Epoch 3 dagi train accuracy -> 0.867

Epoch 3 validation jarayoni tugadi
Epoch 3 dagi validation loss -> 2.192
Epoch 3 dagi validation accuracy -> 0.860
469it [00:10, 43.84it/s]
Epoch 4 train jarayoni tugadi
Epoch 4 dagi train loss -> 2.191
Epoch 4 dagi train accuracy -> 0.872

Epoch 4 validation jarayoni tugadi
Epoch 4 dagi validation loss -> 2.189
Epoch 4 dagi validation accuracy -> 0.882
469it [00:11, 41.34it/s]
Epoch 5 train jarayoni tugadi
Epoch 5 dagi train loss -> 2.189
Epoch 5 dagi train accuracy -> 0.881

Epoch 5 validation jarayoni tugadi
Epoch 5 dagi validation loss -> 2.189
Epoch 5 dagi validation accuracy -> 0.881
469it [00:10, 44.04it/s]
Epoch 6 train jarayoni tugadi
Epoch 6 dagi train loss -> 2.183
Epoch 6 dagi train accuracy -> 0.926

Epoch 6 validation jarayoni tugadi
Epoch 6 dagi validation loss -> 2.179
Epoch 6 dagi validation accuracy -> 0.953
469it [00:11, 41.46it/s]
Epoch 7 train jarayoni tugadi
Epoch 7 dagi train loss -> 2.177
Epoch 7 dagi train accuracy -> 0.966

Epoch 7 validation jarayoni tugadi
Epoch 7 dagi validation loss -> 2.176
Epoch 7 dagi validation accuracy -> 0.970
469it [00:10, 43.58it/s]
Epoch 8 train jarayoni tugadi
Epoch 8 dagi train loss -> 2.177
Epoch 8 dagi train accuracy -> 0.967

Epoch 8 validation jarayoni tugadi
Epoch 8 dagi validation loss -> 2.177
Epoch 8 dagi validation accuracy -> 0.961
469it [00:10, 44.40it/s]
Epoch 9 train jarayoni tugadi
Epoch 9 dagi train loss -> 2.176
Epoch 9 dagi train accuracy -> 0.968

Epoch 9 validation jarayoni tugadi
Epoch 9 dagi validation loss -> 2.177
Epoch 9 dagi validation accuracy -> 0.965
469it [00:10, 43.18it/s]
Epoch 10 train jarayoni tugadi
Epoch 10 dagi train loss -> 2.176
Epoch 10 dagi train accuracy -> 0.970

Epoch 10 validation jarayoni tugadi
Epoch 10 dagi validation loss -> 2.176
Epoch 10 dagi validation accuracy -> 0.972